In [1]:
import os
import requests
import re
import scrapy
import pandas as pd
import numpy as np

In [2]:
#Use this to test xpath/regex
from scrapy.http import TextResponse

# Test URLs:
url1 = 'https://boardgamegeek.com/browse/boardgame/page/1'
url2 = 'https://boardgamegeek.com/browse/boardgame/page/191'
url3 = 'https://boardgamegeek.com/boardgame/174430/gloomhaven/credits'

r = requests.get(url3)
response = TextResponse(r.url, body=r.text, encoding='utf-8')
response

<200 https://boardgamegeek.com/boardgame/174430/gloomhaven/credits>

In [111]:
# dont forget to close the request!
r.close()

In [2]:
# create item used to structure data 
# save this in items.py

import scrapy

class BrowseItem(scrapy.Item):
    board_game_rank = scrapy.Field()
    title = scrapy.Field()
    game_sub_url = scrapy.Field()
    geek_rating = scrapy.Field()
    avg_rating = scrapy.Field()
    num_voters = scrapy.Field()
    pass

class CreditItem(scrapy.Item):
    url = scrapy.Field()
    min_players = scrapy.Field()
    max_players = scrapy.Field()
    min_play_time = scrapy.Field()
    max_play_time = scrapy.Field()
    age = scrapy.Field()
    weight = scrapy.Field()
    #category = scrapy.Field()
    #mechanisms = scrapy.Field()
    #family = scrapy.Field()
    pass

In [ ]:
# save below into CreditSpider.py

import scrapy
from Capstone.items import CreditItem
import requests
import re
import pandas as pd

min_players_pattern = r'minplayers":"\d*'
max_players_pattern = r'maxplayers":"\d*'
min_play_time_pattern = r'minplaytime":"\d*'
max_play_time_pattern = r'maxplaytime":"\d*'
age_pattern = r'minage":"\d*'
weight_dec_pattern = r'"averageweight":\d\.\d*'
weight_pattern = r'"averageweight":\d'
#mechanisms_pattern = r'\\/boardgamemechanic\\/([^\"]+)'
#category_pattern = r'\\/boardgamecategory\\/([^\"]+)'
#family_pattern = r'\\/boardgamefamily\\/([^\"]+)'



df = pd.read_csv('browse_split1.csv')

s = df.full_game_url

class CreditSpider(scrapy.Spider):
    name = "credit"

    def start_requests(self):
        urls = s.tolist()
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        item = CreditItem()
        item['url'] = response.url
        item['min_players'] = re.findall(min_players_pattern, response.text)
        item['max_players'] = re.findall(max_players_pattern, response.text)
        item['min_play_time'] = re.findall(min_play_time_pattern, response.text)
        item['max_play_time'] = re.findall(max_play_time_pattern, response.text)
        item['age'] = re.findall(age_pattern, response.text)
        if re.findall(weight_dec_pattern, response.text) != []:
            item['weight'] = re.findall(weight_dec_pattern, response.text)
        else:
            item['weight'] = re.findall(weight_pattern, response.text)
        #item['category'] = re.findall(category_pattern, response.text)
        #item['mechanisms'] = re.findall(mechanisms_pattern, response.text)
        #item['family'] = re.findall(family_pattern, response.text)
        yield item

In [8]:
# save the below in BrowseSpider.py

import scrapy
from Capstone.items import BrowseItem
import requests

class BrowseSpider(scrapy.Spider):
    name = "browse"

    def start_requests(self):
        urls = ['https://boardgamegeek.com/browse/boardgame/page/1']
        
        # scrapes npages - will work up through page 191 until N/A values become present
        npages = 1

        for i in range(1, npages):
            urls.append("https://boardgamegeek.com/browse/boardgame/page/"+str(i+1)+"")

        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
            n = 0
            y = 0
            while n < 100:
            # each page has 100 board games - this loop ensures all 100 are gathered before going to the next page
                for sel in response.xpath('//table'):
                    item = BrowseItem()
                    item['board_game_rank'] = response.xpath('//tr/td/a[@name]')[n].extract()
                    item['title'] = response.xpath('//tr/td/div/a[@href]/text()')[n].extract()
                    item['game_sub_url'] = response.xpath('//tr/td/div/a/@href')[n].extract()
                    # geek_rating, avg_rating, num_voters all have the same xpath
                    # so y is incremented seperately to collect these fields with their coresponding game
                    item['geek_rating'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y].extract()
                    item['avg_rating'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y+1].extract()
                    item['num_voters'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y+2].extract()
                    n += 1
                    y += 3
                    yield item
           

In [ ]:
# all items for category, mechanism, family not returned in the DOM so selenium is used to find those items
# selenium also can find the other fields from the credits page, but I already wrote the credit spider

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd

cmf_df = pd.DataFrame(columns = ['url', 'category', 'mechanic', 'family'])

url_df = pd.read_csv('browse_split1.csv')

url_s = url_df.full_game_url
urls = url_s.tolist()

options = Options()
options.headless = True

driver = webdriver.Firefox(options=options, executable_path= r'C:\Users\Sam\Desktop\Capstone\Selenium\geckodriver.exe')
driver.implicitly_wait(5) # seconds
print ("Headless Firefox Initialized")

for url in urls:
    driver.get(url)
    print('current url is: ' + str(url))
    
    url_list = []
    url_list.append(url)
    
    category_list = []
    board_game_category = driver.find_elements_by_xpath('//a[contains(@href, "/boardgamecategory/")]')
    for category in board_game_category:
        category_list.append(category.text)
    
    mechanic_list = []
    board_game_mechanic = driver.find_elements_by_xpath('//a[contains(@href, "/boardgamemechanic/")]')
    for mechanic in board_game_mechanic:
        mechanic_list.append(mechanic.text)
    
    family_list = []
    board_game_family = driver.find_elements_by_xpath('//a[contains(@href, "/boardgamefamily/")]')
    for family in board_game_family:
        family_list.append(family.text)
    
    cmf_df = cmf_df.append({'url':url_list, 'category':category_list, 'mechanic':mechanic_list, 'family':family_list}, ignore_index=True)

driver.close()

cmf_df.to_csv('cmf1.csv')

In [ ]:
#execute these in the command line to run the spiders and save data to csv
#scrapy crawl browse -o browse.csv
#scrapy crawl credit -o credit.csv

In [28]:
browse = pd.read_csv('browse.csv')
browse.head()

,avg_rating,board_game_rank,game_sub_url,geek_rating,num_voters,title
0,8.83,"<a name=""1""></a>",/boardgame/174430/gloomhaven,8.573,35473,Gloomhaven
1,8.62,"<a name=""2""></a>",/boardgame/161936/pandemic-legacy-season-1,8.472,37381,Pandemic Legacy: Season 1
2,7.45,"<a name=""201""></a>",/boardgame/38453/space-alert,7.247,14338,Space Alert
3,7.30,"<a name=""701""></a>",/boardgame/95064/ascension-return-fallen,6.695,3080,Ascension: Return of the Fallen
4,7.29,"<a name=""401""></a>",/boardgame/221965/fox-forest,6.969,6974,The Fox in the Forest


In [63]:
# gut check numerical values make sense and that there is the correct number of records
browse.describe()

,avg_rating,geek_rating,num_voters
count,19021.000000,19021.000000,19021.000000
mean,6.386351,5.690198,813.579465
std,0.926287,0.370845,3331.389788
min,1.050000,3.537000,30.000000
25%,5.800000,5.511000,55.000000
50%,6.420000,5.551000,120.000000
75%,7.000000,5.689000,379.000000
max,9.690000,8.573000,94727.000000


In [29]:
# Creates a column 'full_game_url' to extract a list of urls to be used with the second spyder
browse['full_game_url'] = 'https://boardgamegeek.com' + browse['game_sub_url'].astype(str) + "/credits"

In [30]:
browse.head()

,avg_rating,board_game_rank,game_sub_url,geek_rating,num_voters,title,full_game_url
0,8.83,"<a name=""1""></a>",/boardgame/174430/gloomhaven,8.573,35473,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...
1,8.62,"<a name=""2""></a>",/boardgame/161936/pandemic-legacy-season-1,8.472,37381,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...
2,7.45,"<a name=""201""></a>",/boardgame/38453/space-alert,7.247,14338,Space Alert,https://boardgamegeek.com/boardgame/38453/spac...
3,7.30,"<a name=""701""></a>",/boardgame/95064/ascension-return-fallen,6.695,3080,Ascension: Return of the Fallen,https://boardgamegeek.com/boardgame/95064/asce...
4,7.29,"<a name=""401""></a>",/boardgame/221965/fox-forest,6.969,6974,The Fox in the Forest,https://boardgamegeek.com/boardgame/221965/fox...


In [46]:
# extracts numerical rank from column board_game_rank
def clean_rank(rank):
    return re.findall(r'\d+', rank)[0]

browse['board_game_rank_clean'] = browse['board_game_rank'].apply(clean_rank)

# converts numerical rank from object type to int64
browse = browse.astype({'board_game_rank_clean' : 'int64'})

# sorts df by board_game_rank_clean and sets index to the same
browse.sort_values(by='board_game_rank_clean', ignore_index=True, inplace=True)

# rename board_game_rank column
browse.rename(columns={'board_game_rank_clean':'rank'}, inplace=True)

# drop unessesary columns
browse.drop('board_game_rank', axis=1, inplace=True)
browse.drop('game_sub_url', axis=1, inplace=True)

In [101]:
browse.head()

,avg_rating,geek_rating,num_voters,title,full_game_url,rank
0,8.83,8.573,35473,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,1
1,8.62,8.472,37381,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,2
2,8.64,8.279,13199,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,3
3,8.43,8.277,54788,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...,4
4,8.47,8.219,20264,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...,5


In [102]:
browse.to_csv('browse_clean.csv')

In [110]:
# drop columns not needed for the next scrape and 
# split browse into smaller files so I don't have to scrape 20,000 pages all at once

browse_split = browse.drop(['avg_rating','geek_rating','num_voters','title'], axis=1)

browse_split = np.array_split(browse_split, 10)

n = 1
for df in browse_split:
    df.to_csv('browse_split'+str(n)+'.csv')
    n += 1

In [ ]:

# merge creddits with browse data
# in my original credits script the regex pattern did not always
# collect weight as some had decimals and others did not
# this is updated in the current script, but I need to identify which games this is missing for and 
# re run the script for those urls
# my internet also went down during one of my crawls so I will identify those urls as well and rerun them
# occasinally a crawl would miss 1-2 records so those will need to be identified as well

In [2]:
# import credits raw data
credits = pd.DataFrame()

df_list = []
for file in range(1,11):
    df_list.append(pd.read_csv('credit'+str(file)+'.csv'))

credits = pd.concat(df_list)

In [3]:
credits.describe()

,age,max_play_time,max_players,min_play_time,min_players,url,weight
count,17604,17604,17604,17604,17604,17604,14277
unique,21,107,51,85,11,17603,3680
top,"minage"":""12","maxplaytime"":""30","maxplayers"":""4","minplaytime"":""30","minplayers"":""2",https://boardgamegeek.com/boardgame/283864/tra...,"""averageweight"":1.5"
freq,4168,3097,5486,3568,12255,2,553


In [4]:
credits.head()

,age,max_play_time,max_players,min_play_time,min_players,url,weight
0,"minage"":""12","maxplaytime"":""120","maxplayers"":""4","minplaytime"":""60","minplayers"":""1",https://boardgamegeek.com/boardgame/174430/glo...,"""averageweight"":3.8323586744639377"
1,"minage"":""13","maxplaytime"":""60","maxplayers"":""4","minplaytime"":""60","minplayers"":""2",https://boardgamegeek.com/boardgame/161936/pan...,"""averageweight"":2.8296650717703349"
2,"minage"":""14","maxplaytime"":""120","maxplayers"":""4","minplaytime"":""60","minplayers"":""2",https://boardgamegeek.com/boardgame/224517/bra...,"""averageweight"":3.9290322580645163"
3,"minage"":""12","maxplaytime"":""120","maxplayers"":""5","minplaytime"":""120","minplayers"":""1",https://boardgamegeek.com/boardgame/167791/ter...,"""averageweight"":3.2370806890299182"
4,"minage"":""14","maxplaytime"":""120","maxplayers"":""4","minplaytime"":""120","minplayers"":""2",https://boardgamegeek.com/boardgame/182028/thr...,"""averageweight"":4.3949999999999996"


In [3]:
def clean_num_col(num_col):
    return re.findall(r'\d+', num_col)

def clean_num_col_dec(num_col):
    return re.findall('\d\.\d+', str(num_col))

credits['age'] = credits['age'].apply(clean_num_col).str[0].astype(int)
credits['max_play_time'] = credits['max_play_time'].apply(clean_num_col).str[0].astype(int)
credits['max_players'] = credits['max_players'].apply(clean_num_col).str[0].astype(int)
credits['min_play_time'] = credits['min_play_time'].apply(clean_num_col).str[0].astype(int)
credits['min_players'] = credits['min_players'].apply(clean_num_col).str[0].astype(int)
credits['weight'] = credits['weight'].apply(clean_num_col_dec).str[0].astype(np.float16)

In [4]:
credits.head()

,age,max_play_time,max_players,min_play_time,min_players,url,weight
0,12,120,4,60,1,https://boardgamegeek.com/boardgame/174430/glo...,3.832031
1,13,60,4,60,2,https://boardgamegeek.com/boardgame/161936/pan...,2.830078
2,14,120,4,60,2,https://boardgamegeek.com/boardgame/224517/bra...,3.929688
3,12,120,5,120,1,https://boardgamegeek.com/boardgame/167791/ter...,3.236328
4,14,120,4,120,2,https://boardgamegeek.com/boardgame/182028/thr...,4.394531


In [5]:
credits.describe()

,age,max_play_time,max_players,min_play_time,min_players,weight
count,17604.000000,17604.000000,17604.000000,17604.000000,17604.000000,13709.000000
mean,9.646444,96.658544,5.641559,64.926551,2.023177,2.166016
std,3.605177,974.198323,15.916501,201.439763,0.683372,0.790039
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.011719
25%,8.000000,30.000000,4.000000,20.000000,2.000000,1.500000
50%,10.000000,45.000000,4.000000,30.000000,2.000000,2.105469
75%,12.000000,90.000000,6.000000,60.000000,2.000000,2.666016
max,25.000000,120000.000000,999.000000,17280.000000,10.000000,4.914062


In [6]:
browse = pd.read_csv('browse_clean.csv')

In [25]:
browse = browse.merge(credits, how = 'left', left_on = 'full_game_url', right_on = 'url')

In [49]:
browse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19023 entries, 0 to 19022
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     19023 non-null  int64  
 1   avg_rating     19023 non-null  float64
 2   geek_rating    19023 non-null  float64
 3   num_voters     19023 non-null  int64  
 4   title          19023 non-null  object 
 5   full_game_url  19023 non-null  object 
 6   rank           19023 non-null  int64  
 7   age            17587 non-null  float64
 8   max_play_time  17587 non-null  float64
 9   max_players    17587 non-null  float64
 10  min_play_time  17587 non-null  float64
 11  min_players    17587 non-null  float64
 12  url            17587 non-null  object 
 13  weight         13698 non-null  float16
dtypes: float16(1), float64(7), int64(3), object(3)
memory usage: 2.1+ MB


In [50]:
browse_null = browse[browse.isnull().any(axis=1)]

browse_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5325 entries, 224 to 19018
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     5325 non-null   int64  
 1   avg_rating     5325 non-null   float64
 2   geek_rating    5325 non-null   float64
 3   num_voters     5325 non-null   int64  
 4   title          5325 non-null   object 
 5   full_game_url  5325 non-null   object 
 6   rank           5325 non-null   int64  
 7   age            3889 non-null   float64
 8   max_play_time  3889 non-null   float64
 9   max_players    3889 non-null   float64
 10  min_play_time  3889 non-null   float64
 11  min_players    3889 non-null   float64
 12  url            3889 non-null   object 
 13  weight         0 non-null      float16
dtypes: float16(1), float64(7), int64(3), object(3)
memory usage: 592.8+ KB


In [51]:
null_urls = browse_null.url

null_urls.to_csv('null_urls.csv')